# Download using sentinelhub

An sentinelhub account is needed (it is not the same as the CDSE account!)

### Installation

```$ pip install sentinelhub
$ pip install utils```

### Prerequisites

In [2]:
# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

def plot_image(image, factor=1.0, clip_range=None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

### Setting search parameters

In [3]:
start_date = "2022-06-01"
end_date = "2022-06-10"
data_collection = "SENTINEL-2"
aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))'"

In [4]:
json = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z"
).json()
pd.DataFrame.from_dict(json["value"]).head(5)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,3dd25b31-c251-4bc8-acad-1ae00406042d,S2A_MSIL1C_20220606T103631_N0510_R008_T31UES_2...,application/octet-stream,163213055,2024-12-13T17:48:11.031000Z,2025-01-04T04:17:38.361047Z,2025-01-04T04:17:42.974578Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2022/06/06/S2...,"[{'Value': '683a7b2b2cdc92f2d3ef99a5dddcabf8',...","{'Start': '2022-06-06T10:36:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.07401134861950...,"{'type': 'Polygon', 'coordinates': [[[4.074011..."
1,application/octet-stream,3d789bf8-dc7d-412a-92f8-9f9d062355c4,S2A_MSIL2A_20220606T103631_N0510_R008_T31UES_2...,application/octet-stream,209755204,2024-12-13T17:48:28.270000Z,2025-01-04T04:23:42.541013Z,2025-01-04T04:23:48.026044Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/06/06/S2...,"[{'Value': '1183e1783a1cd68759edd7bfac926627',...","{'Start': '2022-06-06T10:36:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.07401134861950...,"{'type': 'Polygon', 'coordinates': [[[4.074011..."
2,application/octet-stream,653eed30-f64f-425d-92ed-01ac4a013be3,S2B_MSIL1C_20220604T104619_N0510_R051_T31UES_2...,application/octet-stream,858137780,2025-02-13T19:45:00.129000Z,2025-03-08T03:40:41.197573Z,2025-03-08T03:40:41.197573Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2022/06/04/S2...,"[{'Value': '6f436f81b5bba012b80f3383ea13eeac',...","{'Start': '2022-06-04T10:46:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((2.99971217585130...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
3,application/octet-stream,fa2184d9-d11c-4678-abf2-05f5d1ffe39a,S2B_MSIL2A_20220604T104619_N0510_R051_T31UES_2...,application/octet-stream,1208976422,2025-02-13T19:45:08.724000Z,2025-03-08T03:45:08.174368Z,2025-03-08T03:45:08.174368Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/06/04/S2...,"[{'Value': 'dbb21b4ef845dfbd176e9ba49c4e9868',...","{'Start': '2022-06-04T10:46:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((2.99971217585130...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
4,application/octet-stream,77f4c6e1-8e77-4749-8236-f8941cd12c87,S2B_MSIL1C_20220601T103629_N0510_R008_T31UES_2...,application/octet-stream,187696287,2024-10-23T16:38:45.413000Z,2024-12-16T17:24:00.811596Z,2024-12-16T17:24:06.940133Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2022/06/01/S2...,"[{'Value': '10e35e7f3c2cefa4a418c1c8b5998d45',...","{'Start': '2022-06-01T10:36:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((4.07166873560485...,"{'type': 'Polygon', 'coordinates': [[[4.071668..."


In [5]:
# Import credentials
# from creds import *


def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]


In [8]:
access_token = get_access_token(
    getpass.getpass("Enter your username"),
    getpass.getpass("Enter your password"),
)

Enter your username········
Enter your password········


In [9]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(acdd7b9a-a5d4-5d10-9ac8-554623b8a0c9)/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

with open("product.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)
            

In [23]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

Enter your SentinelHub client id········
Enter your SentinelHub client secret········


In [25]:
config = SHConfig("cdse")

In [26]:
aoi_coords_wgs84 = [4.20762, 50.764694, 4.487708, 50.916455]

In [27]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

Image shape at 10 m resolution: (1941, 1723) pixels


In [28]:
catalog = SentinelHubCatalog(config=config)

In [29]:
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = "2022-07-01", "2022-07-20"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

InvalidClientError: (invalid_client) Invalid client or Invalid client credentials